# Learning Narrow Passages
### With conditional autoencoder

## Setup

In [1]:
# Import libraries
import torch
from torch.utils.data import TensorDataset, DataLoader
from torch import nn, optim
import numpy as np
import os
from model import CVAE

print("Cuda available?  {}".format(torch.cuda.is_available()))
print("Cuda version: {}".format(torch.version.cuda))
os.environ["CUDA_VISIBLE_DEVICES"] = '0'

Cuda available?  True
Cuda version: 9.0.176


In [2]:
# Options
device = torch.device("cuda:0")
trainProp = 0.8

# Hyperparams: training
batchSize = 256
epochs = 1000
learnRate = 1e-4 

# Hyperparams: network
hiddenEncoderSize = 512
hiddenDecoderSize = 512
latentSize = 3

In [3]:
#Load data
dataFile = "LinearNarrowPassages_Data.npz"
data = np.load(dataFile)
xData = data["x"]
cData = data["c"]
nEntries = xData.shape[0]

# Split into train, test
nTrain = int(nEntries * trainProp)
xTrain = xData[0:nTrain, :]
cTrain = cData[0:nTrain, :]
xTest  = xData[nTrain:, ]
cTest  = cData[nTrain:, ] 

print("Loaded data:")
print("  File: {}".format(dataFile))
print("  Samples (x) shape: {}".format(xData.shape))
print("  Conditions (c) shape: {}".format(cData.shape))
print("  Training: {},   testing: {}".format(xTrain.shape[0], xTest.shape[0]))

Loaded data:
  File: LinearNarrowPassages_Data.npz
  Samples (x) shape: (83731, 6)
  Conditions (c) shape: (83731, 133)
  Training: 66984,   testing: 16747


In [4]:
# Create data loaders
def makeDataLoader(x, c, batchSize):
    xTensor = torch.Tensor(x)
    cTensor = torch.Tensor(c)
    dataset = TensorDataset(xTensor, cTensor)
    dataLoader  = DataLoader(dataset, batch_size=batchSize, shuffle=True)
    return xTensor, cTensor, dataset, dataLoader
    
xTrainTensor, cTrainTensor, trainDataset, trainDataLoader = \
    makeDataLoader(xTrain, cTrain, batchSize)

xTestTensor, cTestTensor, testDataset, testDataLoader = \
    makeDataLoader(xTest, cTest, batchSize)


## Train

In [5]:
# Define loss function
def loss_fn(reconstructed_x, x, mu, logvar, weight):
    # Reconstruction + KL divergence loss 
    
    def weightedMSE(pred, target, weight):
        return (weight * (pred - target) ** 2).mean()
    def MSE(pred, target):
        return torch.sum((pred - target) ** 2)
    
    # Reconstruction loss
    recon_loss = weightedMSE(reconstructed_x, x, weight)
    
    # KL divergence loss
    KL_loss = 10**-4 * 2 * torch.sum(torch.exp(logvar) + mu**2 - 1.0 - logvar, 1)
    
    return torch.mean(recon_loss + KL_loss)

# Define training loop
def train(model, optimizer, dataLoader, epoch, weight, device="cpu"):
    model.train()
    train_loss = 0
    print("")
    for batch_idx, (X, C) in enumerate(dataLoader):
        X, C = X.to(device), C.to(device)
        q_z, logvar, mu, z = model(X, C)
        optimizer.zero_grad()
        W = torch.Tensor(np.tile(weight, (X.shape[0], 1)))
        cvae_loss = loss_fn(q_z, X, mu, logvar, W) 
        cvae_loss.backward()
        train_loss += cvae_loss
        optimizer.step()

    print("====> epoch: {}".format(epoch))    
    print("      last loss: {}".format(cvae_loss))
    print("      mean loss: {:.4f}".format(train_loss / X.shape[0]))
        

In [6]:
# Init model
model = CVAE(
    sample_size = xTrain.shape[1],
    condition_size = cTrain.shape[1],
    hidden_encoder_size = hiddenEncoderSize,
    hidden_decoder_size = hiddenDecoderSize,
    latent_size = latentSize,
).to(device)

# Init optimizer
optimizer = optim.Adam(model.parameters(), lr=learnRate)

# MSE weight
weight = np.array([1.0, 1.0, 1.0, 0.5, 0.5, 0.5])

# Train
for epoch in range(epochs):
    train(model, optimizer, trainDataLoader, epoch, weight, device=device)

/home/ekrell/Documents/Work/repos/LearnedSamplingDistributions/venv/lib/python3.6/site-packages/torch/cuda/__init__.py:118: UserWarning: 
    Found GPU0 GeForce GTX 770 which is of cuda capability 3.0.
    PyTorch no longer supports this GPU because it is too old.
    The minimum cuda capability that we support is 3.5.
    
  warnings.warn(old_gpu_warn % (d, name, major, capability[1]))


RuntimeError: CUDA error: all CUDA-capable devices are busy or unavailable